In [1]:
! pip install scikit-video
import os
import tensorflow as tf	
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

Using TensorFlow backend.


In [ ]:
pframe=40

In [2]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

tf_record_dir = os.path.join('..','tf_records')
os.makedirs(tf_record_dir, exist_ok=True)

tf_record_train = os.path.join(tf_record_dir, 'train' + '.tfrecords')
tf_record_test = os.path.join(tf_record_dir, 'test' + '.tfrecords')
x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
X = np.zeros((x_train.shape[0], pframe, 100, 100,1))
for i in range(x_train.shape[0]):
    if x_train[i].shape[0]>= pframe:
        X[i,: pframe,:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][: pframe,:x_train[i].shape[1],:x_train[i].shape[2]]
    else:
         X[i,:x_train[i].shape[0],:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][:x_train[i].shape[0],:x_train[i].shape[1],:x_train[i].shape[2]]

In [60]:
#X = np.zeros(( 158,212, 100, 100,1))
#for i in range(158):
    #X[i,:x_train[i].shape[0],:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i]


In [3]:
batchsize_video = 1
nb_epoch=50
checkpointer = ModelCheckpoint( filepath=os.path.join('data', 'checkpoints', 'lcrn' + '-' + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    
tb = TensorBoard(log_dir=os.path.join('data', 'logs', 'lcrn'))
early_stopper = EarlyStopping(patience=5)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'lcrn' + '-' + 'training-' + str(timestamp) + '.log'))
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),activation='relu',padding='same'), input_shape=( pframe, 100, 100,1)))
model.add(TimeDistributed(Conv2D(32, (3,3),
kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(2, activation='softmax'))
metrics = ['accuracy']
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
                       metrics=metrics)
model.fit(X, y_train, batch_size=32,  verbose=1,callbacks=[tb, early_stopper, csv_logger], epochs=nb_epoch)




Epoch 1/50
158/158 [==============================] - 73s 462ms/step - loss: 0.7543 - acc: 0.3924
Epoch 2/50


/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


158/158 [==============================] - 71s 447ms/step - loss: 0.7014 - acc: 0.5316
Epoch 3/50
158/158 [==============================] - 72s 457ms/step - loss: 0.7036 - acc: 0.5190
Epoch 4/50
158/158 [==============================] - 72s 454ms/step - loss: 0.6910 - acc: 0.5633
Epoch 5/50
158/158 [==============================] - 70s 442ms/step - loss: 0.6820 - acc: 0.5886
Epoch 6/50
158/158 [==============================] - 73s 459ms/step - loss: 0.6538 - acc: 0.6456
Epoch 7/50
158/158 [==============================] - 76s 479ms/step - loss: 0.7040 - acc: 0.5759
Epoch 8/50
158/158 [==============================] - 73s 460ms/step - loss: 0.6422 - acc: 0.6392
Epoch 9/50
158/158 [==============================] - 72s 455ms/step - loss: 0.6886 - acc: 0.5949
Epoch 10/50
158/158 [==============================] - 73s 463ms/step - loss: 0.6701 - acc: 0.6013
Epoch 11/50
158/158 [==============================] - 71s 448ms/step - loss: 0.6489 - acc: 0.6519
Epoch 12/50
158/158 [========

In [4]:
x_test = get_videos_from_folder(test_folder)

In [5]:
X_test = np.zeros((x_test.shape[0],  pframe, 100, 100,1))
for i in range(x_test.shape[0]):
    if x_test[i].shape[0]>= pframe:
        X[i,: pframe,:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i][: pframe,:x_test[i].shape[1],:x_test[i].shape[2]]
    else:
         X[i,:x_test[i].shape[0],:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i][:x_test[i].shape[0],:x_test[i].shape[1],:x_test[i].shape[2]]

In [9]:
! pip install pandas
train_predictions = model.predict(X_test) 


(69, 2)

In [22]:
dummy_solution = np.array([p[1] for p in train_predictions])
save_solution(my_solution_file,dummy_solution)